In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from llm import config as config_llm
from llm.extract_text import extract
from llm.prepare_dataset import prepare_dataset
from llm.training import LLMTolkien


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/ethan/mambaforge/envs/llm_lotr/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda111.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 111
CUDA SETUP: Loading binary /home/ethan/mambaforge/envs/llm_lotr/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda111.so...


/home/ethan/mambaforge/envs/llm_lotr/lib/python3.11/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/ethan/mambaforge/envs/llm_lotr did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/ethan/mambaforge/envs/llm_lotr/lib/python3.11/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/home/ethan/mambaforge/etc/xml/catalog file'), PosixPath('file')}
  warn(msg)
/home/ethan/mambaforge/envs/llm_lotr/lib/python3.11/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/home/ethan/mambaforge/envs/llm_lotr/lib/python3.11/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories liste

In [3]:
extract(
    url=config_llm.url, 
    start_page=config_llm.start_page, 
    end_page=config_llm.end_page, 
    header_height=config_llm.header_height, 
    footer_height=config_llm.footer_height,
    extraction_path=config_llm.extraction_path
    )

INFO:llm.extract_text:Start extracting pages from https://gosafir.com/mag/wp-content/uploads/2019/12/Tolkien-J.-The-lord-of-the-rings-HarperCollins-ebooks-2010.pdf
INFO:llm.extract_text:Finished extracting texts from https://gosafir.com/mag/wp-content/uploads/2019/12/Tolkien-J.-The-lord-of-the-rings-HarperCollins-ebooks-2010.pdf
INFO:llm.extract_text:Start writing to /home/ethan/llm-tolkien/llm/data/extracted_text.jsonl
100%|██████████| 1210/1210 [01:24<00:00, 14.26it/s]
1011it [01:25, 11.89it/s]
INFO:llm.extract_text:Finished writing to /home/ethan/llm-tolkien/llm/data/extracted_text.jsonl


In [4]:
prepare_dataset(
    dataset_path=config_llm.extraction_path, 
    min_length=config_llm.min_length,
    context_length=config_llm.context_length,
    test_size=config_llm.test_size,
    shuffle=config_llm.shuffle,
    hf_repo=config_llm.hf_repo,
    model_name=config_llm.model_name,
    hf_book_repo=config_llm.hf_book_repo,
    )

INFO:llm.prepare_dataset:Start preparing dataset from /home/ethan/llm-tolkien/llm/data/extracted_text.jsonl


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

INFO:llm.prepare_dataset:The tokenized dataset is composed of 895 elements, each one composed of 2048 tokens.
INFO:llm.prepare_dataset:The training dataset is composed of 805 elements, the test dataset is composed of 90 elements.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

INFO:llm.prepare_dataset:Preparing dataset finished.


In [5]:
lora_config = {
    "r": config_llm.lora_r,
    "lora_alpha": config_llm.lora_alpha,
    "lora_dropout": config_llm.lora_dropout, 
    'bias': config_llm.lora_bias,
    "task_type": config_llm.lora_task_type,
}

trainer_config = {
    "per_device_train_batch_size": config_llm.per_device_train_batch_size, 
    "gradient_accumulation_steps": config_llm.gradient_accumulation_steps,
    "warmup_steps": config_llm.warmup_steps,
    "weight_decay": config_llm.weight_decay,
    "num_train_epochs": config_llm.num_train_epochs,
    "learning_rate": config_llm.learning_rate, 
    "fp16": config_llm.fp16,
    "logging_steps": config_llm.logging_steps, 
    "output_dir": config_llm.hf_repo,
    "overwrite_output_dir": config_llm.overwrite_output_dir,
    "evaluation_strategy": config_llm.evaluation_strategy,
    "save_strategy": config_llm.save_strategy,
    "push_to_hub": config_llm.push_to_hub
}

model = LLMTolkien(config_llm.model_name)
model.train(
    hf_repo=config_llm.hf_repo,
    lora_config=lora_config,
    trainer_config=trainer_config,
    mlm=config_llm.mlm
)

INFO:llm.training:Model trainable parameters:
 trainable params: 4915200 || all params: 3007472640 || trainable%: 0.1634329082375293


EmptyDatasetError: The directory at eturok/llm-tolkien doesn't contain any data files